In [5]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as sg

ROOT = Path('../')
DATA_DIR = ROOT/'data'
OUT_DIR = ROOT/'output'

%load_ext autoreload
%autoreload 2


# Use Google Maps to compute commute matrices

In [6]:
def get_secret(key, secrets_path=ROOT/'secrets.json'):
    """
    Open the JSON file at ``secrets_path``, and return the value corresponding to the given key. 
    """
    secrets_path = Path(secrets_path)
    with secrets_path.open() as src:
        secrets = json.load(src)
    return secrets[key]

GOOGLE_MATRIX_URL = "https://maps.googleapis.com/maps/api/distancematrix/json"
GOOGLE_KEY = get_secret('GOOGLE_API_KEY_ALEX')

def get_matrix(origins, destinations, mode, departure_time=None, url=GOOGLE_MATRIX_URL, 
  key=GOOGLE_KEY, timezone='Pacific/Auckland'):
    """
    Call Google to compute the duration-distance matrix from the list of origins to the 
    list of destinations by the given mode at the given departure time.
    
    INPUT:    
        - origins: list; WGS84 longitude-latitude pairs that will be round to 5 decimal places 
        - destinations: list; WGS84 longitude-latitude pairs that will be round to 5 decimal places 
        - mode: string; one of 'driving', 'walking', 'bicycling', or 'transit'
        - departure_time (optional): string; ISO 8601 datetime string or 'now'; 
          e.g. '2017-06-01T07:30:00'; can't be from the past
        - timezone: string; timezone for query, e.g. 'Pacific/Auckland'; 
          see https://en.wikipedia.org/wiki/List_of_tz_database_time_zones 
            
    OUTPUT:
        A decoded JSON string described at https://developers.google.com/maps/documentation/distance-matrix/intro#DirectionsResponseElements .
    
    NOTES:
        - If ``departure_time`` is not null and ``mode='driving'``, then each request must contain at most 100 elements, 
          where the number of elements equals the product of the number of origins and number of destinations.
    """
    valid_modes = ['driving', 'walking', 'bicycling', 'transit']
    if mode not in valid_modes:
        raise ValueError('mode must be one of {!s}'.format(valid_modes))
    
    origs = '|'.join(["{:.05f},{:.05f}".format(lat, lon) for lon, lat in origins])
    dests = '|'.join(["{:.05f},{:.05f}".format(lat, lon) for lon, lat in destinations])
    if departure_time not in [None, 'now']:
        tz = pytz.timezone(timezone)
        departure_time = dt.datetime.strptime(departure_time, '%Y-%m-%dT%H:%M:%S')
        departure_time = int(tz.localize(departure_time).timestamp())
        
    params = {
        'origins': origs,
        'destinations': dests,
        'key': key,
        'mode': mode,
        'departure_time': departure_time,
    }
    r = requests.get(url, params=params)

    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         

def matrix_to_df(matrix, orig_ids=None, dest_ids=None):
    """
    Given a (decoded) JSON time-distance matrix of the form output by :func:``get_matrix``, 
    a list of origin names (defaults to [0, 1, 2, etc.]), 
    and a list of destination names (defaults to [0, 1, 2, etc.]), convert the matrix to a DataFrame with
    the columns:
    
    - ``'origin'``: one of ``orig_ids``
    - ``'destination'``: one of ``dest_ids``
    - ``'duration'``: time from origin to destination
    - ``'distance'``: distance from origin to destination
    
    The origin and destination names should be listed in the same order as the 'sources' and 'targets' 
    attributes of ``matrix``, respectively.
    """
    # Initialize DataFrame
    columns = ['orig', 'orig_id', 'dest', 'dest_id', 'duration', 'distance']
    f = pd.DataFrame([], columns=columns)
    
    # Append origins and destinations
    origs, dests =  zip(*product(matrix['origin_addresses'], matrix['destination_addresses']))
    f['orig'] = origs
    f['dest'] = dests
    if orig_ids is not None and dest_ids is not None:
        orig_ids, dest_ids = zip(*product(orig_ids, dest_ids))
        f['orig_id'] = orig_ids
        f['dest_id'] = dest_ids
        
    # Append durations and distances
    if 'duration_in_traffic' in matrix['rows'][0]['elements'][0]:
        dur_key = 'duration_in_traffic'
    else:
        dur_key = 'duration'
    durs = []
    dists = []
    for r in matrix['rows']:
        for e in r['elements']:
            if e['status'] == 'OK':
                durs.append(e[dur_key]['value'])
                dists.append(e['distance']['value'])
            else:
                durs.append(None)
                dists.append(None)
    f['duration'] = durs
    f['distance'] = dists

    return f

def collect_matrices(orig_gdf, dest_gdf, orig_id_col, dest_id_col, mode, departure_time=None, 
  skip_self_trips=True, chunk_size=100, url=GOOGLE_MATRIX_URL, key=GOOGLE_KEY):
    """
    Compute the duration-distance matrix between all pairs of rental area points given,
    but skip the diagonal entries, that is, the ones with origin equal to destination.
    To do this, call:func:`get_matrix` repeatedly.
    Group the duration-distance calls into ``chunk_size``-to-1 chunks. 
    
    INPUT:
        - orig_gdf: GeoDataFrame
        - dest_gdf: GeoDataFrame
        - orig_id_col: string; name of ID column in ``orig_gdf``
        - dest_id_col : string; name of ID column in ``dest_gdf``
        - mode: string; see :func:`get_matrix`
        - departure_time: string; see :func:`get_matrix`
        - chunk_size: integer; max number of origin-destination rows per matrix query
        - url: string; see :func:`get_matrix`
        - key : string; see :func:`get_matrix`
            
    OUTPUT:
        A DataFrame of the form...
        
    NOTES:
        - Sleeps for 1 second after every call to :func:`get_matrix` to stay within API usage limits
    """
    odf = orig_gdf.copy()
    ddf = dest_gdf.copy()
    
    # Drop destinations that occur as origins
    if skip_self_trips:
        odf['wkt'] = odf['geometry'].map(lambda x: x.wkt)
        ddf['wkt'] = ddf['geometry'].map(lambda x: x.wkt)
        cond = ~ddf['wkt'].isin(odf['wkt'])
        ddf = ddf[cond].copy()
        del odf['wkt']
        del ddf['wkt']
        
    # Collect matrices
    frames = []
    status = 'OK'
    num_chunks = math.ceil(odf.shape[0]/chunk_size)
    for odfc in np.array_split(odf, num_chunks):
        # Set origins
        origs = [geo.coords[0] for geo in odfc['geometry']] 
        orig_ids = odfc[orig_id_col].values 

        for dest_id, dest_geom in ddf[[dest_id_col, 'geometry']].itertuples(index=False):
            # Quit if bad status
            if status != 'OK':
                print('Quitting because of bad status:', status)
                break
            
            # Setsingle destination
            dests = [dest_geom.coords[0]]  
            dest_ids = [dest_id]
        
            # Get matrix
            try:
                j = get_matrix(origs, dests, mode=mode, departure_time=departure_time, url=url, key=key)
                status = j['status']
                if status != 'OK':
                    break
                df = matrix_to_df(j, orig_ids, dest_ids)
            except:
                df = pd.DataFrame()
                df['orig'] = np.nan
                df['orig_id'] = orig_ids
                df['dest'] = np.nan
                df['dest_id'] = dest_ids
                df['duration'] = np.nan
                df['distance'] = np.nan
            frames.append(df)
            time.sleep(1)
            
    # Concatenate matrices
    return pd.concat(frames).sort_values(['orig', 'dest'])

def compute_google_cost(n, with_freebies=False):
    """
    Estimate the cost of Google matrix query with n elements at 0.5/1000 USD/element 
    beyond 2500 elements.
    If ``with_freebies``, then treat the first 2500 elements as free.
    """
    d = OrderedDict()
    d['#elements needed'] = n
    d['exceeds 100000-element daily limit?'] = n > 100000 
    d['duration for job in minutes'] = (n/100)/60
    if with_freebies:
        d['cost for job in USD'] = (n - 2500)*(0.5/1000)
    else:
        d['cost for job in USD'] = n*(0.5/1000)
    
    return pd.Series(d)

def df_to_gdf(f, lon_col='lon', lat_col='lat'):
    """
    Given a DataFrame with WGS84 longitude values in the column ``lon_col``
    and WGS84 latitude values in the column ``lat_col``, convert it into
    a GeoDataFrame with a geometry column corresponding to the longitude-latitude points.
    Delete the original longitude and latitude columns, and return the result.
    """
    f['geometry'] = f[[lon_col, lat_col]].apply(lambda p: sg.Point(p), axis=1)
    f = f.drop([lon_col, lat_col], axis=1)
    f = gpd.GeoDataFrame(f)
    return f 


In [9]:
# Test some

origs = [
    [174.66339111328125, -36.45000844447082], 
    [174.76158142089844, -36.86533886128865],
    [174.85633850097656, -37.20517535620264],
]
dests = origs[:2]
matrix = get_matrix(origs, dests, mode='driving', departure_time='2017-06-15T08:00:00')
matrix_to_df(matrix, ['bingo', 'bongo', 'boom'], ['bingo', 'bongo'])

,orig,orig_id,dest,dest_id,duration,distance
0,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,0,0
1,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,3827,56385
2,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,2884,53674
3,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,9,0
4,"293-365 Patumahoe Rd, Pukekohe 2120, New Zealand",boom,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,6594,104203
5,"293-365 Patumahoe Rd, Pukekohe 2120, New Zealand",boom,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,4161,54264


In [11]:
# Test some

path = DATA_DIR/'pnr_sites.csv'
pnr = df_to_gdf(pd.read_csv(path))
pnr

collect_matrices(pnr[:2], pnr[1:3], 'site_id', 'site_id', 'driving')

,orig,orig_id,dest,dest_id,duration,distance
0,"1 Thompson Rd, Dakabin QLD 4503, Australia",1,"Gallipoli Way, Kallangur QLD 4503, Australia",3,489,6289
1,"4 English St, Elimbah QLD 4516, Australia",2,"Gallipoli Way, Kallangur QLD 4503, Australia",3,1830,33042


In [17]:
# Load sites

path = DATA_DIR/'pnr_sites.csv'
pnr = df_to_gdf(pd.read_csv(path))
pnr

path = DATA_DIR/'cbd_sites.csv'
cbd = df_to_gdf(pd.read_csv(path))
cbd

path = DATA_DIR/'sa2_centers.csv'
sa2 = df_to_gdf(pd.read_csv(path))
sa2

# Set departure time
departure_time='2017-06-19T09:00:00'


In [40]:
# Collect matrices park and ride sites to park and ride sites

key = get_secret('GOOGLE_API_KEY_PHIL')
mode = 'transit'

n = pnr.shape[0]
print(compute_google_cost(n**2 - n, with_freebies=False))

%time m = collect_matrices(pnr, pnr, 'site_id', 'site_id', mode='transit', departure_time=departure_time, key=key)
n = m.shape[0]
k = m[m['distance'].notnull()].shape[0]
print('hit rate', k/n)
path = OUT_DIR/'pnr_to_pnr_{!s}_matrix_{!s}.csv'.format(mode, departure_time)
m.to_csv(str(path), index=False)




#elements needed for 1 mode(s)           22052
exceeds 100000-element daily limit?      False
duration for job in minutes            3.67533
cost for job in USD                     11.026
dtype: object
CPU times: user 8.04 s, sys: 144 ms, total: 8.18 s
Wall time: 8min 7s
hit rate 0.9999093052784328


In [58]:
# Collect matrices park and ride sites to CBD sites

key = get_secret('GOOGLE_API_KEY_PHIL')
mode = 'transit'

N = pnr.shape[0]*cbd.shape[0]
print(compute_google_cost(N, with_freebies=False))

%time m = collect_matrices(pnr, cbd, 'site_id', 'site_id', mode=mode, departure_time=departure_time, key=key)
n = m.shape[0]
k = m[m['distance'].notnull()].shape[0]
print('hit rate', k/n)
path = OUT_DIR/'pnr_to_cbd_{!s}_matrix_{!s}.csv'.format(mode, departure_time)
m.to_csv(str(path), index=False)




#elements needed for 1 mode(s)               149
exceeds 100000-element daily limit?        False
duration for job in minutes            0.0248333
cost for job in USD                       0.0745
dtype: object
CPU times: user 44 ms, sys: 4 ms, total: 48 ms
Wall time: 3.93 s
hit rate 1.0


In [19]:
# Collect matrices SA2 centers to CDB

key = get_secret('GOOGLE_API_KEY_PHIL')
mode = 'transit'

n = sa2.shape[0]
print(compute_google_cost(n, with_freebies=False))

%time m = collect_matrices(sa2, cbd, 'site_id', 'site_id', mode=mode, departure_time=departure_time, key=key)
n = m.shape[0]
k = m[m['distance'].notnull()].shape[0]
print('hit rate', k/n)
path = OUT_DIR/'sa2_to_cbd_{!s}_matrix_{!s}.csv'.format(mode, departure_time)
m.to_csv(str(path), index=False)




#elements needed                             322
exceeds 100000-element daily limit?        False
duration for job in minutes            0.0536667
cost for job in USD                        0.161
dtype: object
CPU times: user 92 ms, sys: 4 ms, total: 96 ms
Wall time: 7.44 s
hit rate 0.8385093167701864


In [21]:
# Collect matrices SA2 centers to park and ride sites

key = get_secret('GOOGLE_API_KEY_PHIL')
mode = 'driving'

n = sa2.shape[0]*pnr.shape[0]
print(compute_google_cost(n, with_freebies=False))

%time m = collect_matrices(sa2, pnr, 'site_id', 'site_id', mode=mode, departure_time=departure_time, key=key)
n = m.shape[0]
k = m[m['distance'].notnull()].shape[0]
print('hit rate', k/n)
path = OUT_DIR/'sa2_to_pnr_{!s}_matrix_{!s}.csv'.format(mode, departure_time)
m.to_csv(str(path), index=False)




#elements needed                         50876
exceeds 100000-element daily limit?      False
duration for job in minutes            8.47933
cost for job in USD                     25.438
dtype: object
CPU times: user 10.6 s, sys: 292 ms, total: 10.9 s
Wall time: 18min 13s
hit rate 0.9968944099378882
